In [22]:
import requests 
from bs4 import BeautifulSoup
import re
import pandas as pd
from datetime import datetime
import time
from IPython.display import clear_output
import numpy as np 

# OPTIMIZATION : Calculating the start time
start_time = time.time()

#Dictionary for titles urls scraping
title_url_dict= {'title':[]}

#Generating the title list from the 'Electronic Arts' page on metacritic
user_agent = {'User-agent': 'Mozilla/5.0'}
url = 'https://www.metacritic.com/browse/games/score/metascore/all/all/filtered?view=condensed'
response  = requests.get(url, headers = user_agent)
soup = BeautifulSoup(response.text, 'html.parser')

#current page cout 58 (30 items per page)
page_count = int(soup.find('li', class_='page last_page').find('a').text)

for page in range (0,page_count):
        url = 'https://www.metacritic.com/browse/games/score/metascore/all/all/filtered?view=condensed&page='+str(page)
        response  = requests.get(url, headers = user_agent)
        soup = BeautifulSoup(response.text, 'html.parser')

        for title in soup.find_all('td', class_='details'):
                if title.find('a', class_='title') == None:
                        break
                # CLEANUP: Removing mobile titles
                if '/ios/' not in title.find('a', class_='title')['href']:
                        title_url_dict['title'].append(title.find('a')['href'])
        
        # PROGRESSLINE : % of pages processed - printed in the output bar
        clear_output()
        print("Progress: {:2.1%}".format(page / page_count))
        sys.stdout.flush()

#converting the dictionary to a dataframe
title_url_df = pd.DataFrame(title_url_dict)

#generating the titles urls array 
title_url_list = title_url_df['title'].to_numpy()

#generating the platforms and titles names arrays
title_url_df = title_url_df.title.str.split("/",expand=True,)
platforms = title_url_df[2].to_numpy()
titles = title_url_df[3].to_numpy()

# PROGRESSLINE : Completed - printed in the output bar
clear_output()
#print("Complete!")

# OPTIMIZATION : Calculating the time elapsed
print("--- %s seconds ---" % (time.time() - start_time))

--- 287.98885703086853 seconds ---


In [23]:
# The function below converts the date from datetime format inot an integer in the 20201231 format
def to_integer(dt_time):
    return 10000*dt_time.year + 100*dt_time.month + dt_time.day

In [24]:
# OPTIMIZATION : Calculating the start time
start_time = time.time()

user_agent = {'User-agent': 'Mozilla/5.0'}

#Dictionary for titles info scraping
ea_title_dict = {'DATE_ID':[], 'PUBLISHER_URL':[], 'TITLE_URL':[], 'TITLE_NAME':[], 'PLATFORM':[],'TITLE_GENRES':[], 'RELEASE_DATE_ID':[],
                'METASCORE':[], 'CRITIC_REVIEWS_COUNT':[],'POS_CRITIC_REVIEWS_COUNT':[],'MIX_CRITIC_REVIEWS_COUNT':[],'NEG_CRITIC_REVIEWS_COUNT':[],
                'AGV_USER_SCORE':[], 'USER_RATINGS_COUNT':[],'POS_USER_REVIEWS_COUNT':[],'MIX_USER_REVIEWS_COUNT':[],'NEG_USER_REVIEWS_COUNT':[]}

#Iterating through the list of titles to scrape the info
for i in range(0,len(title_url_list)):
        
        url = 'https://www.metacritic.com' + title_url_list[i]
        response  = requests.get(url, headers = user_agent)
        
        # Checking that the page exists
        if response.status_code == 200: 
           
            soup = BeautifulSoup(response.text, 'html.parser')
            url_base = title_url_list[i]
            
            # CLEANUP - Removing titles with no metacritic score or Canceled
            critic_score = soup.find('a', class_='metascore_anchor', href=url_base+'/critic-reviews')
            release_date = soup.find('li', class_='summary_detail release_data').find('span', class_='data').text

            if critic_score != None and release_date != 'Canceled':

                # Today's date, title url, title name, platform & metascore
                Today_Date = int(time.strftime("%Y%m%d"))

                ea_title_dict['DATE_ID'].append(Today_Date)
                ea_title_dict['TITLE_URL'].append(title_url_list[i])
                ea_title_dict['TITLE_NAME'].append(titles[i])
                ea_title_dict['PLATFORM'].append(platforms[i])
                ea_title_dict['METASCORE'].append(int(critic_score.find('span').text))

                # publisher_url
                publisher_url = soup.find('li', class_='summary_detail publisher')
                if publisher_url == None:
                    ea_title_dict['PUBLISHER_URL'].append('')
                else:
                    ea_title_dict['PUBLISHER_URL'].append(publisher_url.find('a')['href'].replace('/company/',''))

                #average user score
                agv_user_score = soup.find('a', class_='metascore_anchor', href=url_base+'/user-reviews')
                if agv_user_score == None or agv_user_score.find('div').text == 'tbd':
                    ea_title_dict['AGV_USER_SCORE'].append(float(0))
                else:
                    ea_title_dict['AGV_USER_SCORE'].append(float(agv_user_score.find('div').text))
                
                #release_date
                release_date = soup.find('li', class_='summary_detail release_data')
                if release_date == None:
                    ea_title_dict['RELEASE_DATE_ID'].append(int(0))
                else:
                    #exporting the date in the datetime format or keeping the original format for dates that do not fit the format
                    try :
                        # Date Format 'Oct 13, 2020'
                        release_date = datetime.strptime(release_date.find('span', class_='data').text,'%b %d, %Y')
                        ea_title_dict['RELEASE_DATE_ID'].append(to_integer(release_date))
                    except ValueError:
                        try:
                            # Date Format '2020'
                            release_date = datetime.strptime(release_date.find('span', class_='data').text,'%Y')
                            ea_title_dict['RELEASE_DATE_ID'].append(to_integer(release_date))
                        except ValueError:
                            try:
                                # Date Format 'September 2020'
                                release_date = datetime.strptime(release_date.find('span', class_='data').text,'%B %Y')
                                ea_title_dict['RELEASE_DATE_ID'].append(to_integer(release_date))
                            except ValueError:
                                # Other Date Formats stored as 0
                                ea_title_dict['RELEASE_DATE_ID'].append(int(0))
            
                #List of genres separated by comma
                title_genres_text = ''

                # CLEANUP - Reducing the number of genres to 18 (as defined by Metacritic) + Miscellaneous 
                title_genres_list = ['Action','Adventure','Action Adventure','Fighting','First-Person','Flight','Party','Platformer','Puzzle',
                                    'Racing','Real-Time','Role-Playing','Simulation','Sports','Strategy','Third-Person','Turn-Based','Wargames','Wrestling','Miscellaneous']

                title_genres = soup.find('li', class_='summary_detail product_genre')
                
                if title_genres == None:
                    ea_title_dict['TITLE_GENRES'].append('')
                else:
                    for genre in title_genres.find_all('span', class_='data'):
                            # CLEANUP - Removing duplicated genre names
                            if genre.text in title_genres_list and genre.text not in title_genres_text:
                                if title_genres_text == '':
                                    title_genres_text = title_genres_text+genre.text
                                else:
                                    title_genres_text = title_genres_text+','+genre.text
                    if title_genres_text == '':
                        ea_title_dict['TITLE_GENRES'].append('Miscellaneous')
                    else:
                        ea_title_dict['TITLE_GENRES'].append(title_genres_text)

                #number of critic reviews and number of user reviews
                critic_reviews = 0
                user_reviews = 0

                for summary in soup.find_all('div', class_='summary'):
                        if summary.find('a', href=url_base+'/critic-reviews') != None:
                            critic_reviews = summary.find('a', href=url_base+'/critic-reviews').find('span').text
                            ea_title_dict['CRITIC_REVIEWS_COUNT'].append(int(critic_reviews))
                        
                        if summary.find('a', href=url_base+'/user-reviews') != None:
                            user_reviews = summary.find('a', href=url_base+'/user-reviews').text
                            ea_title_dict['USER_RATINGS_COUNT'].append(int(user_reviews.replace(' Ratings','')))

                if critic_reviews == 0:
                            ea_title_dict['CRITIC_REVIEWS_COUNT'].append(int(0))
                if user_reviews == 0:
                            ea_title_dict['USER_RATINGS_COUNT'].append(int(0))
            
                url_base_count = url_base +'/critic-reviews?dist='

                #number of positive critic reviews
                pos_critic_reviews_count = soup.find('a', href=url_base_count+'positive')
                if pos_critic_reviews_count == None:
                    ea_title_dict['POS_CRITIC_REVIEWS_COUNT'].append(int(0))
                else:
                    ea_title_dict['POS_CRITIC_REVIEWS_COUNT'].append(int(pos_critic_reviews_count.find('span', class_="count").text))
                
                #number of neutral critic reviews
                mix_critic_reviews_count = soup.find('a', href=url_base_count+'neutral')
                if mix_critic_reviews_count == None:
                    ea_title_dict['MIX_CRITIC_REVIEWS_COUNT'].append(int(0))
                else:
                    ea_title_dict['MIX_CRITIC_REVIEWS_COUNT'].append(int(mix_critic_reviews_count.find('span', class_="count").text))
                
                #number of negative critic reviews
                neg_critic_reviews_count = soup.find('a', href=url_base_count+'negative')
                if neg_critic_reviews_count == None:
                    ea_title_dict['NEG_CRITIC_REVIEWS_COUNT'].append(int(0))
                else:
                    ea_title_dict['NEG_CRITIC_REVIEWS_COUNT'].append(int(neg_critic_reviews_count.find('span', class_="count").text))
            
                url_base_count = url_base +'/user-reviews?dist='
                
                #number of positive user reviews
                pos_user_reviews_count = soup.find('a', href=url_base_count+'positive')
                if pos_user_reviews_count == None:
                    ea_title_dict['POS_USER_REVIEWS_COUNT'].append(int(0))
                else:
                    ea_title_dict['POS_USER_REVIEWS_COUNT'].append(int(pos_user_reviews_count.find('span', class_="count").text.replace(',','')))
                
                #number of neutral user reviews
                mix_user_reviews_count = soup.find('a', href=url_base_count+'neutral')
                if mix_user_reviews_count == None:
                    ea_title_dict['MIX_USER_REVIEWS_COUNT'].append(int(0))
                else:
                    ea_title_dict['MIX_USER_REVIEWS_COUNT'].append(int(mix_user_reviews_count.find('span', class_="count").text.replace(',','')))
                
                #number of negative user reviews
                neg_user_reviews_count = soup.find('a', href=url_base_count+'negative')
                if neg_user_reviews_count == None:
                    ea_title_dict['NEG_USER_REVIEWS_COUNT'].append(int(0))
                else:
                    ea_title_dict['NEG_USER_REVIEWS_COUNT'].append(int(neg_user_reviews_count.find('span', class_="count").text.replace(',','')))

            # Progress - % of titles processed - printed in the output bar
            clear_output()
            print("Progress: {:2.1%}".format(i / len(title_url_list))+" ")
            sys.stdout.flush()

# Progress - Completed - printed in the output bar
clear_output()
#print("Complete!")

# OPTIMIZATION : Calculating the time elapsed
print("--- %s seconds ---" % (time.time() - start_time))

Progress: 80.8% 


ConnectionError: ('Connection aborted.', OSError("(60, 'ETIMEDOUT')"))

In [25]:
#Converting the dictionary into the dataframe
ea_title_df = pd.DataFrame(ea_title_dict, columns = ['DATE_ID','PUBLISHER_URL','TITLE_URL', 'TITLE_NAME', 'PLATFORM','TITLE_GENRES', 'RELEASE_DATE_ID',
                        'METASCORE', 'CRITIC_REVIEWS_COUNT','POS_CRITIC_REVIEWS_COUNT','MIX_CRITIC_REVIEWS_COUNT','NEG_CRITIC_REVIEWS_COUNT',
                        'AGV_USER_SCORE', 'USER_RATINGS_COUNT', 'POS_USER_REVIEWS_COUNT','MIX_USER_REVIEWS_COUNT','NEG_USER_REVIEWS_COUNT'])

In [26]:
# The function below is used to break down the titles per row per genre

def explode(df, lst_cols, fill_value='', preserve_index=False):
    # make sure `lst_cols` is list-alike
    if (lst_cols is not None
        and len(lst_cols) > 0
        and not isinstance(lst_cols, (list, tuple, np.ndarray, pd.Series))):
        lst_cols = [lst_cols]
    # all columns except `lst_cols`
    idx_cols = df.columns.difference(lst_cols)
    # calculate lengths of lists
    lens = df[lst_cols[0]].str.len()
    # preserve original index values    
    idx = np.repeat(df.index.values, lens)
    # create "exploded" DF
    res = (pd.DataFrame({
                col:np.repeat(df[col].values, lens)
                for col in idx_cols},
                index=idx)
             .assign(**{col:np.concatenate(df.loc[lens>0, col].values)
                            for col in lst_cols}))
    # append those rows that have empty lists
    if (lens == 0).any():
        # at least one list in cells is empty
        res = (res.append(df.loc[lens==0, idx_cols], sort=False)
                  .fillna(fill_value))
    # revert the original index order
    res = res.sort_index()
    # reset index if requested
    if not preserve_index:        
        res = res.reset_index(drop=True)
    return res

In [27]:
# Transform the column 'titles_genres from string into a list
for i in range (0, len(ea_title_df)):
    ea_title_df['TITLE_GENRES'][i] = list(ea_title_df['TITLE_GENRES'][i].split(","))

# Apply function above on the df
ea_title_df = explode(ea_title_df, ['TITLE_GENRES'], fill_value='', preserve_index=True)
ea_title_df = ea_title_df[['DATE_ID','PUBLISHER_URL','TITLE_URL', 'TITLE_NAME', 'PLATFORM','TITLE_GENRES', 'RELEASE_DATE_ID',
                        'METASCORE', 'CRITIC_REVIEWS_COUNT','POS_CRITIC_REVIEWS_COUNT','MIX_CRITIC_REVIEWS_COUNT','NEG_CRITIC_REVIEWS_COUNT',
                        'AGV_USER_SCORE', 'USER_RATINGS_COUNT', 'POS_USER_REVIEWS_COUNT','MIX_USER_REVIEWS_COUNT','NEG_USER_REVIEWS_COUNT']]

In [28]:
# Importing the required packages for all your data framing needs.
import pandas as pd

# The Snowflake Connector library.
import snowflake.connector as snow
from snowflake.connector.pandas_tools import write_pandas

## Phase I: Truncate/Delete the current data in the table
# The connector...
conn = snow.connect(user="LCUZACOV@EA.COM",
   authenticator='externalbrowser',
   account="eagai.us-east-1",
   role="SUPERUSERS_GAI",
   # (the prefix in your snowflake space... for example, 
   # company.snowflakecomputing.com would just be "company" as the ACCOUNT name)
   warehouse="QUERY",
   database= "GAITWAY_SANDBOX",
   schema="BF5_SANDBOX")

Initiating login request with your identity provider. A browser window should have opened for you to complete the login. If you can't see it, check existing browser windows, or your OS settings. Press CTRL+C to abort and try again...


DatabaseError: 250001 (08001): Failed to connect to DB: eagai.us-east-1.snowflakecomputing.com:443. IP 188.148.244.170 is not allowed to access Snowflake.  Contact your local security administrator.

In [29]:
cur = conn.cursor()
sql = "use role SUPERUSERS_GAI"
cur.execute(sql)
sql = "use database GAITWAY_SANDBOX"
cur.execute(sql)
sql = "use schema BF5_SANDBOX"
cur.execute(sql)

cur = conn.cursor()

sql = "truncate table if exists LCU_EA_METACRITIC"
cur.execute(sql)

#Close the cursor.
cur.close()
write_pandas(conn, ea_title_df, "LCU_ALL_METACRITIC")

ProgrammingError: 390420: IP 188.148.244.170 is not allowed to access Snowflake.  Contact your local security administrator.

In [30]:
#Exporting the the dataframe into an EXCEL file with the date and time in the title
#Date_Time = time.strftime("%Y%m%d-%H%M%S")
#excelfilename = 'all_metacritic_'+Date_Time +".xlsx"
#ea_title_df.to_excel (r'/Users/liuba/Desktop/files/'+excelfilename, index = False, header=True)

In [31]:
#Exporting the the dataframe into an CSV file with the date and time in the title
#Date_Time = time.strftime("%Y%m%d-%H%M%S")
#excelfilename = 'all_metacritic_'+Date_Time +".csv"
#ea_title_df.to_csv (r'/Users/liuba/Desktop/files/'+excelfilename, index = False, header=True)